In [10]:
import os
import re
import json
import pandas as pd
import numpy as np
from functools import partial
from IPython.core.debugger import set_trace
from tqdm import tqdm_notebook
from konlpy.tag import Okt
from multiprocessing import Pool, Process

In [27]:
pd.read_pickle('twitter_data/aldo (2010.01.01-2019.02.07).pkl').droplevel(0, axis=1)

,id,text
0,424094692545150976,http://ift.tt/1hluftb sunny!!!오늘뱅쿠버날씨짱따듯..♡♡ ...
1,665816281694752768,한여름밤의꿈 e ourtros 31 seguiram você.Rumo aos 12....
2,665072806250442753,하이올라 http://www.hihola.co.kr 해외구매대행쇼핑몰 ALDO ...
3,480341446751289345,[Facebook update] 140621 가로수길 #ALDO 오픈행사해요! 저도...
4,477203073966673920,#강남 http://ift.tt/SE4k81 http://ift.tt/1q8Ylq...
5,476051667125682176,#알도 #aldo #알도매장 #강남역 #앤트윤 #antyoon #beautifulo...
6,473398446322249728,"칼퇴+택시로 크록스, 알도 다 와봤지만 찾고있는 신발은 모두 품절 중... ㅠㅠ #..."
7,458017584474882048,"""@adamlambert: #Aldo Lamberti -- my Italian co..."
8,444365352588173312,알도VS라마스 알도 스탠딩 전략 동영상 http://goo.gl/fb/uKTd5 ...
9,444264867030380544,#강남역 #강남역 #알도 #ALDO 우오아 http://ift.tt/1ko9SNE ...


In [18]:
pd.read_pickle('twitter_data/allsaints (2010.01.01-2019.02.07).pkl').columns.levels[0][0]

'allsaints'

In [7]:
srcdir = 'twitter_data'
textcol = 'text'
fnames = [srcdir + '/' + fname for fname in os.listdir(srcdir)];fnames

['twitter_data/aldo (2010.01.01-2019.02.07).pkl',
 'twitter_data/allsaints (2010.01.01-2019.02.07).pkl',
 'twitter_data/crocs (2010.01.01-2019.02.07).pkl',
 'twitter_data/freitag (2010.01.01-2019.02.07).pkl',
 'twitter_data/salomon (2010.01.01-2019.02.07).pkl']

In [20]:
tokenize(fnames[0], textcol)

[['sunny',
  '오늘',
  '뱅쿠버',
  '날씨',
  '짱',
  '따다',
  '♡♡',
  'daily',
  'dailylook',
  'ootd',
  'jean',
  'sweater',
  'balenciagara',
  '7',
  'jean',
  'aldo',
  '데',
  '..',
  '.pic.twitter.com/gsarhpds4c'],
 ['한',
  '여름밤',
  '꿈',
  'e',
  'ourtros',
  '31',
  'seguiram',
  'aos',
  '12.700',
  'seguidores',
  'gppetrobrasf',
  '1',
  'ronda',
  'omundoseriamelhorse',
  'joanna',
  'aldo',
  'mudabrasil',
  'spfc'],
 ['하이',
  '오르다',
  '해외',
  '구매',
  '대행',
  '쇼핑몰',
  'aldo',
  '알도',
  '슈즈',
  '세',
  '일',
  '정보',
  '11',
  '윌',
  '16일',
  '까지',
  '전',
  '품목',
  '30%',
  'off',
  '신발',
  '여성',
  '신발',
  '남성',
  '신발',
  'aldo'],
 ['facebook',
  'update',
  '140621',
  '가로수길',
  '오픈',
  '행사',
  '해',
  '저',
  '받다',
  '문신',
  '아니다',
  '이벤트',
  '용',
  '헤나',
  'aldo',
  'seoyoungjin97thpic.twitter.com/8spo9unz5i'],
 ['iamhanbonnie',
  '오빠',
  '선물',
  '해주다',
  '두번째',
  '신발',
  '나다',
  '하나',
  '지르다',
  '버리다',
  '느낌',
  '강남',
  'aldo',
  '커플',
  '신발'],
 ['이쁘다',
  '사이즈',
  '없다',
  'aldo',
  '알

In [70]:
class Sentences:
    def __init__(self, fnames, textcol):
        self.fnames = fnames
        self.textcol = textcol
        self.length = 0
        
    def __iter__(self):
        for fname in self.fnames: 
            df = pd.read_pickle(fname)
            item = df.columns.levels[0][0]
            #yield from df.droplevel(0, axis=1)[self.textcol]
        
            for doc in df.droplevel(0, axis=1)[self.textcol]:
                preprocessed = preproc(doc)
                text, hashtags = preprocessed['text'], preprocessed['hashtags']
                text += ' ' + ' '.join(hashtags)
                yield text


    def __len__(self):
        if self.length == 0:
            for fname in self.fnames:
                self.length += len(pd.read_pickle(fname))
            
        return self.length

In [71]:
sentences = Sentences(fnames, 'text')

In [72]:
from soynlp.word import WordExtractor

word_extractor = WordExtractor(
    min_frequency=5,
    min_cohesion_forward=0.05, 
    min_right_branching_entropy=0.0
)

word_extractor.train(sentences)
words = word_extractor.extract()

training was done. used memory 0.138 Gbry 0.137 Gb
all cohesion probabilities was computed. # words = 3517
all branching entropies was computed # words = 7926
all accessor variety was computed # words = 7926


In [116]:
iter_sent = iter(sentences)

In [127]:
next(iter_sent)

'iamhanbonnie 오빠에게 선물해준 두번째 신발  나도 하나 질러버려서  느낌으로 강남 aldo 커플신발'

In [128]:
preproc(pd.read_pickle('twitter_data/aldo (2010.01.01-2019.02.07).pkl').droplevel(0,axis=1).text.iloc[4])

{'urls': ['http://ift.tt/se4k81',
  'http://ift.tt/1q8ylqz',
  '..pic.twitter.com/u6btdf62t3'],
 'hashtags': ['강남', 'aldo', '커플신발'],
 'mention': [],
 'text': 'iamhanbonnie 오빠에게 선물해준 두번째 신발  나도 하나 질러버려서  느낌으로'}

In [129]:
pd.read_pickle('twitter_data/aldo (2010.01.01-2019.02.07).pkl').droplevel(0,axis=1).text.iloc[4]

'#강남 http://ift.tt/SE4k81\xa0 http://ift.tt/1q8Ylqz\xa0 iamhanbonnie 오빠에게 선물해준 두번째 신발 #ALDO 나도 하나 질러버려서 #커플신발 느낌으로..pic.twitter.com/u6BTDf62T3'

In [115]:
import math

def word_score(score):
    return (score.cohesion_forward * math.exp(score.right_branching_entropy))

print('단어   (빈도수, cohesion, branching entropy)\n')
for word, score in sorted(words.items(), key=lambda x:word_score(x[1]), reverse=True)[:100]:
    print('%s     (%d, %.3f, %.3f)' % (
            word, 
            score.leftside_frequency, 
            score.cohesion_forward,
            score.right_branching_entropy
            )
         )

단어   (빈도수, cohesion, branching entropy)

freitag     (355, 0.945, 3.552)
크록스     (880, 0.989, 3.152)
..     (30, 0.714, 3.280)
프라이탁     (244, 0.877, 3.000)
살로몬     (121, 0.943, 2.805)
알도     (165, 0.859, 2.801)
crocs     (601, 0.914, 2.736)
allsaints     (449, 0.941, 2.693)
...     (15, 0.598, 2.860)
!!     (7, 0.700, 2.690)
윤아     (780, 0.996, 2.206)
ㅋㅋ     (61, 0.910, 2.210)
올세인츠     (319, 0.953, 1.986)
korea     (26, 0.789, 2.160)
daily     (51, 0.799, 2.077)
헨리     (45, 1.000, 1.838)
신발     (140, 0.505, 2.500)
에서     (34, 0.425, 2.656)
^^     (15, 1.000, 1.714)
ㅎㅎ     (19, 1.000, 1.667)
aldo     (107, 0.527, 2.283)
행사     (43, 0.796, 1.855)
회사     (30, 1.000, 1.600)
yoona     (313, 0.957, 1.638)
샌들     (27, 0.818, 1.787)
re     (39, 0.443, 2.373)
너무     (41, 0.837, 1.721)
브랜드     (28, 0.907, 1.609)
컬렉션     (37, 0.887, 1.627)
salomon     (71, 0.736, 1.799)
ㅋㅋㅋ     (35, 0.723, 1.796)
함께     (49, 0.942, 1.517)
캠페인     (29, 0.874, 1.587)
같은     (19, 0.613, 1.934)
글로벌     (29, 0.983, 1.

In [19]:
def preproc(text, url=True, mention=True, hashtag=True, remove=True):
    LINEBREAK = r'\n' # str.replace에서는 r'\n'으로 검색이 안된다
    RT = ' rt '
    EMOJI = r'[\U00010000-\U0010ffff]'
    
    #URL = r'(?P<url>(https?://)?(www[.])?[^ \u3131-\u3163\uac00-\ud7a3]+[.][a-z]{2,6}\b([^ \u3131-\u3163\uac00-\ud7a3]*))'
    URL = r'(?:https?:\/\/)?(?:www[.])?[^ :\u3131-\u3163\uac00-\ud7a3]+[.][a-z]{2,6}\b(?:[^ \u3131-\u3163\uac00-\ud7a3]*)'
    # \u3131-\u3163\uac00-\ud7a3 는 한글을 의미함
    HASHTAG = r'(#[^ #@]+)'
    MENTION = r'(@[^ #@]+)' 
    
    PTNS = '|'.join((LINEBREAK, RT, URL, HASHTAG, MENTION, EMOJI))
    
    out = {}
    text = re.sub('|'.join((LINEBREAK,RT)), '', text.lower())
    
    if url:
        urls = re.findall(URL, text) #[m.groupdict()['url'] for m in re.finditer(URL, text)]
        urls = [url.replace('\xa0', '') for url in urls]
        out['urls'] = urls
        
    if hashtag:
        hashtags = re.findall(HASHTAG, text)
        out['hashtags'] = [h[1:] for h in hashtags]
        
    if mention:
        mentions = re.findall(MENTION, text)
        out['mention'] = [m[1:] for m in mentions]
        
    if remove:    
        text = re.sub(PTNS, '', text)
        
    out['text'] = text.strip()
    return out


tagger = Okt()

def tokenize(fname, textcol):
    df = pd.read_pickle(fname)
    item = df.columns.levels[0][0]
    df = df.droplevel(0, axis=1)
    tokenized = []
    
    for doc in tqdm_notebook(df[textcol].iloc[:]):
        preprocessed = preproc(doc)
        text, hashtags = preprocessed['text'], preprocessed['hashtags']
        text += ' ' + ' '.join(hashtags)
        _tokenized = [t[0] for t in tagger.pos(text, norm=True, stem=True) if t[1] not in ['Punctuation', 'Josa']]
        tokenized.append(_tokenized)
    
    #out = df[['id']].copy()
    
    #df['brand'] = file.split('.')[0]
    #out['tokenized'] = tokenized
    #return out
    return tokenized

In [6]:
preproc(pd.read_pickle('twitter_data/allsaints (2010.01.01-2019.02.07).pkl').droplevel(0,axis=1).text.iloc[210], remove=True)

{'urls': ['http://www.fashionn.com/board/read_new.php?table=1006&number=21030&sns_number=989523…',
  'pic.twitter.com/2rys36ronu'],
 'hashtags': ['올세인츠', '올세인츠비치컬렉션pic.twitter.com/2rys36ronu'],
 'mention': [],
 'text': "올세인츠,  여름 한정판 '헌팅턴 비치 컬렉션' 출시"}